In [5]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [3]:
# Database connection information
user = 'dsbc_student'
pw = '7*.8G9QH21'
host = '142.93.121.174'
port = '5432'
db = 'weatherinszeged'

In [4]:
# Establish and dispose connection
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, pw, host, port, db))

weather = pd.read_sql_query('select * from weatherinszeged', con=engine)

engine.dispose()

In [6]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
date                   96453 non-null datetime64[ns, UTC]
summary                96453 non-null object
preciptype             96453 non-null object
temperature            96453 non-null float64
apparenttemperature    96453 non-null float64
humidity               96453 non-null float64
windspeed              96453 non-null float64
windbearing            96453 non-null float64
visibility             96453 non-null float64
loudcover              96453 non-null float64
pressure               96453 non-null float64
dailysummary           96453 non-null object
dtypes: datetime64[ns, UTC](1), float64(8), object(3)
memory usage: 8.8+ MB


In [7]:
# Independent and dependent variables
X = weather[['humidity', 'windspeed', 'windbearing', 'pressure']]
Y = weather[['temperature']]

In [9]:
# Linear Regression model
lrm = LinearRegression()
lrm.fit(X, Y)
print('Coefficients: \n', lrm.coef_)
print('Intercept: \n', lrm.intercept_)

Coefficients: 
 [[-3.24962454e+01 -2.01413742e-01  4.04836936e-03 -6.91759209e-04]]
Intercept: 
 [37.92643819]


### Gauss Markov Linear Assumptions 